The following cell holds the definition of our parameters,
these values can be overriden by rendering the with e.g. the following command:

```bash
papermill -p alpha 0.2 -p ratio 0.3 universe_analysis.ipynb output/test_run.ipynb
```


In [1]:
run_no = 0
universe_id = "test"
universe = {
    "model": "logreg", # "rf", "gbm"
    "cutoff": ["quantile_0.15", "quantile_0.3"],
    "exclude_features": "none", # "frau1", "maxdeutsch1", "maxdeutsch.Missing."
}
output_dir="./output"
seed=0

In [2]:
import json
# Parse universe into dict if it is passed as a string
if isinstance(universe, str):
    universe = json.loads(universe)

In [3]:
# Auto-reload the custom package
%load_ext autoreload
%autoreload 1
%aimport fairness_multiverse


In [4]:
from fairness_multiverse.universe import UniverseAnalysis

universe_analysis = UniverseAnalysis(
    run_no = run_no,
    universe_id = universe_id,
    universe = universe,
    output_dir=output_dir,
)

Always use the same seed


In [5]:
import numpy as np
parsed_seed = int(seed)
np.random.seed(parsed_seed)
print(f"Using Seed: {parsed_seed}")

Using Seed: 0


## Loading Data

In [6]:
from pathlib import Path
import pandas as pd

# Define the path to the data
data_path = 'Kern24_code/data/siab.csv'

# Use custom caching of data (optional if you want to cache the synthetic data)
#cache_dir = Path("data")
#cache_dir.mkdir(exist_ok=True)
#cache_file = cache_dir / "synthetic_dataset.csv.gz"

if Path(data_path).exists():
    print(f"Loading data from: {data_path}")
    dataset = pd.read_csv(data_path)
else:
    raise FileNotFoundError(f"Data file not found at {data_path}")

# Optionally, cache the loaded dataset (if you want to save it for future use)
#dataset.to_csv(cache_file, index=False)

Loading data from: Kern24_code/data/siab.csv


In [7]:
dataset

,id,year,dummy,ltue,frau1,maxdeutsch1,maxdeutsch.Missing.,f3,f4,f5,...,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159
0,1,2010,0.374540,0,1,0,0,-0.234153,-0.234137,1.579213,...,0.346448,-0.680025,0.232254,0.293072,-0.714351,1.865775,0.473833,-1.191303,0.656554,-0.974682
1,2,2010,0.746491,0,1,0,0,0.787085,-0.867641,0.145862,...,1.912318,1.065227,-1.792045,-0.446774,1.204923,-1.196660,0.770678,0.667056,-0.699147,-0.008817
2,3,2010,0.441531,0,1,1,0,0.412931,-0.563725,-0.822220,...,-1.422254,-0.646573,-1.081548,1.687142,0.881640,-0.007973,1.479944,0.077368,-0.861284,1.523124
3,4,2010,0.127580,0,0,1,0,0.538910,-1.913778,0.941285,...,1.080460,-1.844905,-1.017192,0.516669,-0.799111,-0.164536,0.000699,0.158256,-0.677566,0.149287
4,5,2010,0.209906,1,1,1,0,-0.032753,-0.543425,-0.712846,...,-0.894607,-0.186872,-0.439731,1.446978,0.196555,1.031845,-1.485560,0.267050,0.889631,0.082284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279995,279996,2016,0.059231,0,0,0,0,0.789103,1.098175,0.575662,...,1.371849,0.837904,-0.593873,0.023140,-0.527913,-1.767966,-0.728683,-0.458071,0.602688,-0.605510
279996,279997,2016,0.343070,0,1,1,1,1.307429,-0.333882,1.871347,...,-0.901311,2.533277,-0.582527,0.551837,-0.216997,1.810090,-0.799598,-0.985467,-0.678710,-0.081515
279997,279998,2016,0.893136,0,0,1,0,-0.193517,-0.174826,1.621908,...,-1.234041,-0.463881,-0.279631,-0.148189,0.581454,-0.326943,2.262004,2.661323,-0.320801,1.395938
279998,279999,2016,0.763567,0,1,1,0,1.186777,0.834553,-0.943291,...,0.559828,0.072501,0.269816,0.548500,-0.999140,-0.040551,2.574617,-1.324465,0.359664,1.024497


## Preprocessing Data

```
# Invert the categories dictionary
categories_inverted = {
    column: {v: k for k, v in mapping.items()}
    for column, mapping in categories.items()
}
```


### Exclude Protected Features

In [ ]:
# Note this will always be n >= 1, even if empty!
excluded_features = universe["exclude_features"].split("-")
excluded_features_dictionary = {
    "race": "RAC1P",
    "sex": "SEX",
    "immigration": "NATIVITY",
}

# Code nice names to column names
excluded_feature_columns = [
    excluded_features_dictionary[f] for f in excluded_features if len(f) > 0 and f != "none"
]

if len(excluded_feature_columns) > 0:
    print(f"Dropping features: {excluded_feature_columns}")
    features.drop(excluded_feature_columns, axis=1, inplace=True)


### Continuous Variables: Binning / Log-Scaling / Keeping Them As-Is

In [ ]:
from sklearn.pipeline import make_pipeline
from fairness_multiverse.universe import preprocess_continuous

transformer_age, bins_age = preprocess_continuous(source_data=features, column_name="AGEP", configuration=universe["preprocess_age"])
transformer_income, bins_income = preprocess_continuous(source_data=features, column_name="PINCP", configuration=universe["preprocess_income"])

continuous_processor = make_pipeline(
    transformer_age,
    transformer_income
)

### Categorical Variables: One-Hot or Ordinal Encoding

In [ ]:
from sklearn.pipeline import TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

all_categorical_columns = list(set(categories.keys()).intersection(set(features.columns)))

# For which columns is ordinal encoding even an option?
categorical_columns_to_transform = [
    'SCHL',
    # 'MAR',
    # 'SEX',
    # 'DIS',
    # 'ESP',
    # 'CIT',
    # 'MIG',
    'MIL',
    # 'ANC',
    # 'NATIVITY',
    # 'DEAR',
    # 'DEYE',
    # 'DREM',
    # 'ESR',
    # 'ST',
    # 'FER',
    # 'RAC1P'
]

# Support to-be-binned continuous variables
def add_binned_variable_to_categorical_transformation(colname, values):
    if values is not None:
        categorical_columns_to_transform.append(colname)
        categories[colname] = {val: val for val in values}

add_binned_variable_to_categorical_transformation("AGEP", bins_age)
add_binned_variable_to_categorical_transformation("PINCP", bins_income)

def nested_list(all_categories, columns_to_use):
    categories = { col: all_categories[col] for col in columns_to_use }
    # Create a nested list from the categories dict
    categories_list = [[v for k, v in mapping.items()] for column, mapping in categories.items()]
    return categories_list

if (universe["encode_categorical"] == "ordinal"):
    categorical_transformer = OrdinalEncoder(
        categories = nested_list(categories, categorical_columns_to_transform),
    )
elif (universe["encode_categorical"] == "one-hot"):
    categorical_transformer = OneHotEncoder(
        categories = nested_list(categories, categorical_columns_to_transform),
        sparse_output=False
    )
else:
    raise "Unsupported universe option for encode_categorical"

# One-Hot Encode all other cateogircal columns
other_categorical_columns = list(set(all_categorical_columns) - set(categorical_columns_to_transform))
other_transformer = OneHotEncoder(
    categories = nested_list(categories, other_categorical_columns),
    sparse_output=False
)

categorical_preprocessor = ColumnTransformer([
        ("encode_categorical", categorical_transformer, categorical_columns_to_transform),
        ("encode_categorical_rest", other_transformer, other_categorical_columns),
    ],
    remainder='passthrough',
)

## Split Data


In [ ]:
# Select stratification strategy
if universe["stratify_split"] == "none":
    stratify = None
elif universe["stratify_split"] == "target":
    stratify = label
elif universe["stratify_split"] == "protected-attribute":
    stratify = features_org["RAC1P"]
elif universe["stratify_split"] == "both":
    # Concatinate both columns
    stratify = features_org["RAC1P"].astype(str) + "-" + label["PUBCOV"].astype(str)

stratify

In [ ]:
from sklearn.model_selection import train_test_split

(
    X_train, X_test,
    y_train, y_true,
    group_train, group_test,
    org_train, org_test
) = train_test_split(
    features,
    label,
    group,
    features_org,
    test_size=0.2,
    # Note: The analysis originally used two distinct seeds, one for numpy (defaulting to 2023) and one for the train_test_split (defaulting to 0).
    # To allow for exact reproducibility of the original results, as well as specification of only a single seed we base this second seed off the first one.
    # If you adapt this code for your own analysis feel free to remove this line and replace it e.g. with e.g. a call to numpy.random.randint.
    random_state=abs(parsed_seed - 2023),
    stratify=stratify
)

## Post-Splitting Processing

If e.g. only train data is affected

### Exclude Certain Subgroups

In [ ]:
# Extract configuration
exclude_subgroups_config = universe["exclude_subgroups"].split("_")
if len(exclude_subgroups_config) == 1:
    exclude_subgroups_config = (exclude_subgroups_config[0], None, None)
excl_subgroups_method, excl_subgroup_colname, excl_subgroups_value = exclude_subgroups_config

if excl_subgroup_colname == "race":
    excl_subgroup_column = "RAC1P"
    excl_subgroup_counts = org_train[excl_subgroup_column].value_counts()
elif excl_subgroups_method != "keep-all":
    raise Exception("Unsupported configuration for exclude_subgroups:" + universe["exclude_subgroups"])

if excl_subgroups_method == "keep-all":
    # Don't need to do anything
    excl_subgroup_column = None
    excl_subgroup_values = []
else:
    if excl_subgroups_method == "drop-smallest":
        drop_smallest_n = int(excl_subgroups_value)
        excl_subgroup_values = list(excl_subgroup_counts.tail(drop_smallest_n).index)
    elif excl_subgroups_method == "keep-largest":
        keep_largest_n = int(excl_subgroups_value)
        excl_subgroup_values = list(excl_subgroup_counts.tail(
            len(excl_subgroup_counts) - keep_largest_n
        ).index)
    elif excl_subgroups_method == "drop-name":
        excl_subgroup_values = [excl_subgroups_value]
    else:
        raise Exception("Unsupported configuration for exclude_subgroups:" + universe["exclude_subgroups"])

    if excl_subgroup_column is not None:
        print(f"Dropping values: {excl_subgroup_values}")
        keep_rows_mask = ~org_train[excl_subgroup_column].isin(excl_subgroup_values)

    n_rows_to_drop = (~keep_rows_mask).sum()
    if n_rows_to_drop > 0:
        print(f"Dropping N = {n_rows_to_drop} ({n_rows_to_drop / len(keep_rows_mask):.2%}) rows from {excl_subgroup_colname}")
        X_train = X_train[keep_rows_mask]
        y_train = y_train[keep_rows_mask]
        group_train = group_train[keep_rows_mask]

## Fitting the Model

Select which model to fit

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

if (universe["model"] == "logreg"):
    model = LogisticRegression()
elif (universe["model"] == "rf"):
    model = RandomForestClassifier()
elif (universe["model"] == "svm"):
    model = SVC()
elif (universe["model"] == "gbm"):
    model = GradientBoostingClassifier()
elif (universe["model"] == "elasticnet"):
    model = LogisticRegression(penalty = 'elasticnet', solver = 'saga', l1_ratio = 0.5)
else:
    raise "Unsupported universe.model"

In [ ]:
from fairness_multiverse.universe import predict_w_threshold

In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

model = Pipeline([
    ("continuous_processor", continuous_processor),
    ("categorical_preprocessor", categorical_preprocessor),
    ("scale", StandardScaler() if universe["scale"] == "scale" else None),
    ("model", model),
])

model.fit(X_train, y_train)

y_prob = model.predict_proba(X_test)
y_pred_default = predict_w_threshold(y_prob, 0.5)

# Naive prediction
accuracy_score(y_true = y_true, y_pred = y_pred_default)

In [ ]:
# Evaluate
model.predict(X_test)

## (Fairness) Metrics

- Using [Fairlearn](https://fairlearn.org/v0.8/quickstart.html)


In [ ]:
import numpy as np

colname_to_bin = "RAC1P"
majority_value = features_org[colname_to_bin].mode()[0]

org_test["majmin"] = np.where(org_test[colname_to_bin] == majority_value, "majority", "minority")

The greater of two metrics: `true_positive_rate_difference` and
`false_positive_rate_difference`. The former is the difference between the
largest and smallest of $P[h(X)=1 | A=a, Y=1]$, across all values :math:`a`
of the sensitive feature(s). The latter is defined similarly, but for
$P[h(X)=1 | A=a, Y=0]$.
The equalized odds difference of 0 means that all groups have the same
true positive, true negative, false positive, and false negative rates. [src](https://fairlearn.org/main/api_reference/generated/fairlearn.metrics.equalized_odds_difference.html)

> This shouldn't differ based on which class we see as "good" or "bad"?

In [ ]:
example_universe = universe.copy()
example_universe["cutoff"] = example_universe["cutoff"][0]
example_universe["eval_fairness_grouping"] = example_universe["eval_fairness_grouping"][0]
fairness_dict, metric_frame = universe_analysis.compute_metrics(
    example_universe,
    y_pred_prob=y_prob,
    y_test=y_true,
    org_test=org_test,
)

### Overall

#### Fairness

Main fairness target: **Equalized Odds**.
Seems to be a better fit than equal opportunity, since we're not only interested in Y = 1.
Seems to be a better fit than demographic parity, since we also care about accuracy, not just equal distribution of preds.


Pick column for computation of fairness metrics

#### Performance

Overall performance measures, most interesting in relation to the measures split by group below


In [ ]:
metric_frame.overall

### By Group


In [ ]:
metric_frame.by_group

Graphical comparison:

In [ ]:
# In a graphic
metric_frame.by_group.plot.bar(
    subplots=True,
    layout=[3, 3],
    legend=False,
    figsize=[12, 8],
    title="Show all metrics",
)

### Equality of opportunity violation

```python
white_tpr = np.mean(y_pred[(y_true == 1) & (group_test == 1)])
black_tpr = np.mean(y_pred[(y_true == 1) & (group_test == 2)])

white_tpr - black_tpr
```

## Final Output


In [ ]:
sub_universes = universe_analysis.generate_sub_universes()
len(sub_universes)

In [ ]:
import pandas as pd

# Uses excl_subgroup_column and values from the global scope

def filter_sub_universe_data(sub_universe, org_test):
  # Generate an all True mask to start with
  keep_rows_mask = np.ones(org_test.shape[0], dtype=bool)

  # Potentially remove any subgroups from the test set
  if (sub_universe["eval_exclude_subgroups"] == "exclude-in-eval"):
    if excl_subgroup_column is not None:
      assert excl_subgroup_values is not None

      exclude_subgroup_eval_mask = ~org_test[excl_subgroup_column].isin(excl_subgroup_values)
      keep_rows_mask = keep_rows_mask & exclude_subgroup_eval_mask

      n_rows_to_drop = (~exclude_subgroup_eval_mask).sum()
      print(f"[drop subgroups] Dropping N = {n_rows_to_drop} ({n_rows_to_drop / len(keep_rows_mask):.2%}) rows from {excl_subgroup_colname}")
  elif (sub_universe["eval_exclude_subgroups"] == "keep-in-eval"):
    pass
  else:
    raise "Unsupported eval_exclude_subgroups"

  # Potentially use a smaller and more "convenient" subset of the data to do evalaution on
  if (sub_universe["eval_on_subset"] == "full"):
    pass
  else:
    if sub_universe["eval_on_subset"].startswith("locality"):
      # Filter based on locality / region
      # Step 1: Decide which regions to keep
      if (sub_universe["eval_on_subset"] == "locality-largest-only"):
        # Use the largest PUMA region
        puma_regions_to_keep = [org_test["PUMA"].value_counts().idxmax()]
      elif (sub_universe["eval_on_subset"] == "locality-whitest-only"):
        # Find the majority class on the prot. attribute
        majority_class = org_test["RAC1P"].value_counts().index[0]
        majority_class

        # Find the PUMA region with the highest share of the majority class
        counts = pd.DataFrame()
        counts["full"] = org_test["PUMA"].value_counts(sort=False)
        counts["majority"] = org_test[org_test["RAC1P"] == majority_class]["PUMA"].value_counts(sort=False)
        counts["fraction"] = counts["majority"] / counts["full"]

        # Use the PUMA region with the highest share of the majority class
        majority_puma_id = counts.sort_values(by="fraction", ascending=False).index[0]
        puma_regions_to_keep = [majority_puma_id]
      elif (sub_universe["eval_on_subset"] == "locality-city-la"):
        puma_regions_to_keep = list(range(3701, 3769+1))
      elif (sub_universe["eval_on_subset"] == "locality-city-sf"):
        puma_regions_to_keep = list(range(7501, 7507+1))

      # Step 2: Keep only those regions
      print(f"Keeping the following PUMA regions: {puma_regions_to_keep}")
      eval_on_subset_mask = org_test["PUMA"].isin(puma_regions_to_keep)
    elif (sub_universe["eval_on_subset"] == "exclude-military"):
      # Only keep non-military personnel
      eval_on_subset_mask = (org_test["MIL"].isin(["Never served in the military", "N/A (less than 17 years old)"]))
    elif (sub_universe["eval_on_subset"] == "exclude-non-citizens"):
      # Only keep US citizens
      eval_on_subset_mask = ~(org_test["CIT"] == "Not a citizen of the U.S.")
    else:
      raise "Unsupported eval_on_subset"

    keep_rows_mask = keep_rows_mask & eval_on_subset_mask

    n_rows_to_drop = (~eval_on_subset_mask).sum()
    print(f"[subset] Dropping N = {n_rows_to_drop} ({n_rows_to_drop / len(keep_rows_mask):.2%}) rows")

  n_rows_to_drop = (~keep_rows_mask).sum()
  print(f"[TOTAL] Dropping N = {n_rows_to_drop} ({n_rows_to_drop / len(keep_rows_mask):.2%}) rows. Final size: {keep_rows_mask.sum()}.")

  return keep_rows_mask

In [ ]:
final_output = universe_analysis.generate_final_output(
    y_pred_prob=y_prob,
    y_test=y_true,
    org_test=org_test,
    save=True,
    filter_data=filter_sub_universe_data
)
final_output